In [2]:
#일반 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#API를 사용하기 위한 import
# from geoband.API import *
# import folium
# import json
# import geopandas as gpd

#model
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression


In [3]:
police11 = pd.read_csv("police11.csv")
police11 = police11.iloc[:,1:]
police11.head()

jur_stn  year  상반기  파출소수  경찰관수  5대범죄발생건  5대범죄검거건   5대범죄검거율    전체인구   전체성비  \
0  수원남부경찰서  2017    0     9   395     3184     1417  0.445038  405665  102.4   
1  수원남부경찰서  2017    1     9   395     2967     1426  0.480620  405665  102.4   
2  수원남부경찰서  2018    0     9   395     3143     1378  0.438435  406281  102.3   
3  수원남부경찰서  2018    1     9   395     2825     1374  0.486372  406281  102.3   
4  수원남부경찰서  2019    0     9   395     2589     1725  0.666280  404995  102.3   

   ...    교통신고비율    질서신고비율  tot_f_safety  dept_safety  crime_safety  \
0  ...  0.126613  0.341020          73.6         73.1          79.2   
1  ...  0.126068  0.343160          71.9         70.9          78.3   
2  ...  0.138606  0.323678          74.1         74.0          80.5   
3  ...  0.118676  0.349347          69.0         68.2          75.5   
4  ...  0.139994  0.309494          74.9         74.7          78.8   

    rob_satety_q1  mur_safety_q2  ta_safety_q3  raw_odder_q4  overall_q5  
0            78.2           80.2          68.9          70.5        74.7  
1            78.4           78.2          69.1          64.7        74.1  
2            80.7           80.2          67.4          73.3        74.4  
3            74.0           76.9          62.4          65.7        71.0  
4            77.6           79.9          71.2          73.8        75.5  

[5 rows x 43 columns]

# 교통사고 데이터

In [213]:
df = pd.read_csv("11.5개시도_교통사고현황.csv")
# date 전처리
df["date"] = df["date"].str.rstrip("월")

# year
df["year"] = df["date"].str[:4] 

# 상반기
temp = df["date"].str[6:].replace(["1","2","3","4","5","6","7","8","9"],["01","02","03","04","05","06","07","08","09"], inplace= False)
df["상반기"] = (temp.str[:] <= "06").astype("int")

# 교통사고 발생건수
traffic = df.groupby(["jur_stn","year","상반기"]).count().reset_index()

# 마지막 전처리
traffic["jur_stn"] = traffic["jur_stn"].str[:] + "경찰서"
traffic = traffic.iloc[:,:4]
traffic = traffic.rename(columns = {"date":"교통사고발생건"})
traffic = traffic.astype({"year":"int"})
traffic["jur_stn"] = traffic["jur_stn"].str[:].replace("진해경찰서경찰서", "진해경찰서")
traffic = traffic[traffic["year"].isin([2018,2019])]

print(traffic.shape)
traffic.head()

(164, 4)


jur_stn  year  상반기  교통사고발생건
0  마산동부경찰서  2018    0      186
1  마산동부경찰서  2018    1      203
2  마산동부경찰서  2019    0      243
3  마산동부경찰서  2019    1      213
6  마산중부경찰서  2018    0      308

In [214]:
police12 = pd.merge(police11, traffic, how = "outer", on = ["jur_stn","year","상반기"])

# 2017년 데이터를 회귀분석으로 결측치 채우기

In [215]:
dataset = police12.drop(["jur_stn","year","상반기"], axis = 1)
train, test = dataset[dataset["교통사고발생건"].isnull() == False], dataset[dataset["교통사고발생건"].isnull()]
X_train, Y_train = train.drop(["교통사고발생건"], axis = 1), train["교통사고발생건"]
X_test = test.drop(["교통사고발생건"], axis = 1)

print(X_train.shape, Y_train.shape)
print(X_test.shape)

(164, 40) (164,)
(82, 40)


In [216]:
model = LinearRegression(normalize= True)
model.fit(X_train, Y_train)
pred = model.predict(X_test)

In [217]:
police12.loc[police12["교통사고발생건"].isnull(),"교통사고발생건"] = pred

# 신속출동까지 걸린 시간 데이터를 포함시키기

#### 2017 & 2018 & 2019 데이터로 만들기

서울 [243,260,255] <br>
세종 [np.nan,np.nan, 357] <br>
경기남부(수원) [352, 335, 332] <br>
경남(창원, 진주) [370, 359, 365] <br>


In [218]:
police_name = sorted(police12["jur_stn"].str[:].unique().tolist())        

In [219]:
### 통합버전
action = [370, 359, 365]*2 # 마산 2개
action += [243,260,255]*31
action += [np.nan, np.nan, 357] #세종
action += [352, 335, 332]*3 # 수원
action += [370, 359, 365]*4 # 진주, 진해, 창원서부/창원중부

act = pd.DataFrame(sorted(police_name*3))
act["year"] = [2017,2018,2019]*41
act["평균출동시간"] = action
act = act.rename(columns = {0: "jur_stn"})
print(act.shape)
act.head()

(123, 3)


jur_stn  year  평균출동시간
0  마산동부경찰서  2017   370.0
1  마산동부경찰서  2018   359.0
2  마산동부경찰서  2019   365.0
3  마산중부경찰서  2017   370.0
4  마산중부경찰서  2018   359.0

In [220]:
police12 = pd.merge(police12, act, how = "inner", on = ["jur_stn","year"])


# 평균출동시간 결측치를 회귀분석으로 처리하기

In [221]:
dataset = police12.drop(["jur_stn","year","상반기"], axis = 1)
train, test = dataset[dataset["평균출동시간"].isnull() == False], dataset[dataset["평균출동시간"].isnull()]
X_train, Y_train = train.drop(["평균출동시간"], axis = 1), train["평균출동시간"]
X_test = test.drop(["평균출동시간"], axis = 1)

print(X_train.shape, Y_train.shape)
print(X_test.shape)

(242, 41) (242,)
(4, 41)


In [222]:
model = LinearRegression(normalize= True)
model.fit(X_train, Y_train)
pred = model.predict(X_test)

In [223]:
pred

array([368.28170268, 369.50202186, 372.28958805, 372.06129991])

In [224]:
police12.loc[police12["평균출동시간"].isnull(),"평균출동시간"] = pred

### 저장전에 전처리 및 columns 순서

In [225]:
# type
police12 = police12.astype({"교통사고발생건": "int",
                            "평균출동시간": "int"})
# 순서
police12 = pd.concat([police12.iloc[:,:35], police12[["교통사고발생건","평균출동시간"]], police12.iloc[:,35:-2]], axis = 1)


police12.to_csv("police12.csv")

In [226]:
police12 = pd.concat([police12.iloc[:,:35], police12[["교통사고발생건","평균출동시간"]], police12.iloc[:,35:-2]], axis = 1)
